Примеры последовательностей для выравнивания с семинара:

In [1]:
seq1 = "ATCGTAGC"
seq2 = "ATCGTACG"
seq3 = "GCTAGCTA"
seq4 = "TACGATCG"
rna1 = "AUGCUAGC"
rna2 = "AUGCUGCA"
rna3 = "GCUAGCUA"
rna4 = "UACGAUCG"
protein1 = "ACDEFGHIKLMNPQRSTVWY"
protein2 = "ACDFGHIKLMNPRSTVWY"
protein3 = "ACDEFGHLMNPQRSTVWX"
protein4 = "ABCDEFGHIJKLMN"

Расстояние Хемминга - простое вычисление количества отличающихся нуклеотидов (аминокислот). 
Реализация для простого случая с одинаковой длиной сиквенсов:

In [2]:
def Hemming_distance(seq1, seq2):
    l = len(seq1)
    f = 0
    for i in range(l):
        if seq1[i] != seq2[i]:
            f += 1
    return f

In [3]:
print(Hemming_distance(seq1, seq2))

2


Расстояние Левенштейна - минимальное количество односимвольных операций (вставки, пропуски, замены), необходимых для превращения одной последовательности символов в другую.

In [4]:
def Levenshtain_distance(seq1, seq2):
    n, m = len(seq1), len(seq2)
    if n > m:
        seq1, seq2 = seq2, seq1
        n, m = m, n
    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if seq1[j - 1] != seq2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [5]:
print(Levenshtain_distance(seq1, seq2))

2


Алгоритм Нидлмана-Вунша - глобальное выравнивание последовательностей с учетом штрафов за несовпадения и введение разрывов (пропусков).

In [6]:
def Needleman_Wunsch(seq1, seq2, match=1, mismatch=-1, indel=-2):
    n, m = len(seq1), len(seq2)
    matrix = [[0] * (m+1) for _ in range(n+1)]

    for i in range(1, n+1):
        matrix[i][0] = matrix[i-1][0] + indel
    for j in range(1, m+1):
        matrix[0][j] = matrix[0][j-1] + indel
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            diag = matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            up = matrix[i-1][j] + indel
            left = matrix[i][j-1] + indel
            matrix[i][j] = max(diag, up, left)
    
    align1, align2 = [], []
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and j > 0 and matrix[i][j] == matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch):
            align1.append(seq1[i-1])
            align2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif i > 0 and matrix[i][j] == matrix[i-1][j] + indel:
            align1.append(seq1[i-1])
            align2.append('-')
            i -= 1
        else:
            align1.append('-')
            align2.append(seq2[j-1])
            j -= 1
    return ''.join(reversed(align1)), ''.join(reversed(align2)), matrix[n][m], matrix

Использование алгоритма для построения выравнивания на выданных последовательностях и скорах

In [14]:
alig1, alig2, score, matrix = Needleman_Wunsch("CGTCTT", "CATTCT", match=1, mismatch=-1, indel=-2)

print(f"Выравнивание последовательности 1: {alig1}\nВыравнивание последовательности 2: {alig2}\nИтоговый скор: {score}")

matrix

Выравнивание последовательности 1: CGTCTT
Выравнивание последовательности 2: CATTCT
Итоговый скор: 0


[[0, -2, -4, -6, -8, -10, -12],
 [-2, 1, -1, -3, -5, -7, -9],
 [-4, -1, 0, -2, -4, -6, -8],
 [-6, -3, -2, 1, -1, -3, -5],
 [-8, -5, -4, -1, 0, 0, -2],
 [-10, -7, -6, -3, 0, -1, 1],
 [-12, -9, -8, -5, -2, -1, 0]]

Алгоритм Смифа-Ватермана - локальное выравнивание, схоже с предыдущим алгоритмом.

In [25]:
def Smith_Waterman(seq1, seq2, match_score=1, gap_score=1, mismatch_score=2):

    n, m = len(seq1), len(seq2)
    matrix = [[0] * (m + 1) for _ in range(n + 1)]  
    max_score = 0  
    max_pos = (0, 0)  

    for i in range(1, n + 1):
        for j in range(1, m + 1):

            match = matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else -mismatch_score)
            delete = matrix[i - 1][j] - gap_score
            insert = matrix[i][j - 1] - gap_score
            matrix[i][j] = max(0, match, delete, insert)  

            if matrix[i][j] > max_score:
                max_score = matrix[i][j]
                max_pos = (i, j)

    align1, align2 = '', ''
    i, j = max_pos
    while matrix[i][j] > 0:
        if matrix[i][j] == matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else -mismatch_score):
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif matrix[i][j] == matrix[i - 1][j] - gap_score:
            align1 = seq1[i - 1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j - 1] + align2
            j -= 1

    return align1, align2, max_score

In [26]:
seq1 = "GATTACA"
seq2 = "GCATGCU"
alignment1, alignment2, score = Smith_Waterman("CGTCTT", "CATTCT")
print("Выравнивание последовательности 1:", alignment1)
print("Выравнивание последовательности 2:", alignment2)
print("Оценка:", score)

Выравнивание последовательности 1: TCT
Выравнивание последовательности 2: TCT
Оценка: 3


Использование библиотек из Python:

In [27]:
#pip install Bio

In [28]:
from Bio import Align
from Bio.Align import PairwiseAligner

aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = 1
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -2

seq1 = "CGTCTT"
seq2 = "CATTCT"

alignments = aligner.align(seq1, seq2)
for alignment in alignments:
    print(alignment)

target            0 CGTCTT 6
                  0 |.|..| 6
query             0 CATTCT 6

